In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import jieba
import joblib

# Load data from CSV
data_path = "data/test_sample.csv"
df = pd.read_csv(data_path)

# Define aspect categories
aspect_columns = [col for col in df.columns if col not in ['index', 'reviewbody', 'star']]

# Text preprocessing with Chinese segmentation
def preprocess_text(text):
    words = jieba.cut(text)
    return ' '.join(words)

# Prepare features
df['processed_review'] = df['reviewbody'].apply(preprocess_text)

# Inspect processed_review
print("\nProcessed reviews (first 3 samples):")
for i in range(min(3, len(df))):
    print(f"\nRow {i}:")
    print(f"Original: {df['reviewbody'].iloc[i]}")
    print(f"Processed: {df['processed_review'].iloc[i]}")

X = df['processed_review']

# Prepare target variables - Keep as DataFrame
y = df[aspect_columns].astype('object')

# Convert sentiment scores to categorical labels
def convert_sentiment(score):
    if score == -2:
        return 'not_mentioned'
    elif score == -1:
        return 'negative'
    elif score == 0:
        return 'neutral'
    elif score == 1:
        return 'positive'



# Apply conversion to each column with .loc
for col in y.columns:
    y.loc[:, col] = y[col].apply(convert_sentiment)

# Text vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Verify shapes
print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Train model for each aspect with class checking
classifiers = {}
for aspect in aspect_columns:
    unique_classes = y_train[aspect].nunique()
    if unique_classes < 2:
        print(f"Skipping {aspect}: only one class ({y_train[aspect].iloc[0]}) in training data")
        classifiers[aspect] = None
    else:
        clf = LinearSVC(random_state=42)
        clf.fit(X_train, y_train[aspect])
        classifiers[aspect] = clf

# Evaluation
print("\nModel Evaluation:")
for aspect in aspect_columns:
    if classifiers[aspect] is not None:
        y_pred = classifiers[aspect].predict(X_test)
        y_true = y_test[aspect]
        print(f"\n{aspect}:")
        print(classification_report(y_true, y_pred))
    else:
        print(f"\n{aspect}: No model trained (single class in training data)")

# Function to predict sentiments for new review
def predict_sentiments(review_text):
    processed_text = preprocess_text(review_text)
    text_tfidf = tfidf.transform([processed_text])
    
    predictions = {}
    for aspect, clf in classifiers.items():
        if clf is None:
            predictions[aspect] = 'not_mentioned'  # Default for single-class aspects
        else:
            pred = clf.predict(text_tfidf)[0]
            predictions[aspect] = pred
    
    return predictions

# Example prediction
new_review = "这家餐厅环境很好，服务态度也不错，菜品味道很棒，就是价格有点贵，停车不太方便。"
predictions = predict_sentiments(new_review)
print("\nPredictions for new review:")
for aspect, sentiment in predictions.items():
    print(f"{aspect}: {sentiment}")

# # Save the model
# joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
# for aspect, clf in classifiers.items():
#     if clf is not None:
#         joblib.dump(clf, f'classifier_{aspect}.pkl')


Processed reviews (first 3 samples):

Row 0:
Original: 和老公的朋友们聚餐，选择了这家烧肉屋，据说人气很旺，于是定座的，我们坐在楼上，阁楼感觉有点空气不流通，我这个孕妇貌似不适合油烟味大的地方，虽然已经过了孕吐期，但一开始不太习惯，发现窗没有可以开的，都是固定的有点小失望，后来慢慢就习惯了，开始点了几个甜筒，很大一个，味道还凑合，后来大家点了很多肉，其中有一个肉味道很好，但是只顾着吃也忘记叫啥了，点了两份石锅饭也全干掉了，朋友的老婆点了冷面，第一次吃这样的，感觉不错啊！
Processed: 和 老公 的 朋友 们 聚餐 ， 选择 了 这家 烧肉 屋 ， 据说 人气 很旺 ， 于是 定座 的 ， 我们 坐在 楼上 ， 阁楼 感觉 有点 空气 不 流通 ， 我 这个 孕妇 貌似 不 适合 油烟味 大 的 地方 ， 虽然 已经 过 了 孕吐 期 ， 但 一 开始 不太 习惯 ， 发现 窗 没有 可以 开 的 ， 都 是 固定 的 有点 小 失望 ， 后来 慢慢 就 习惯 了 ， 开始 点 了 几个 甜筒 ， 很大 一个 ， 味道 还 凑合 ， 后来 大家 点 了 很多 肉 ， 其中 有 一个 肉 味道 很 好 ， 但是 只顾 着 吃 也 忘记 叫 啥 了 ， 点 了 两份 石锅 饭 也 全 干掉 了 ， 朋友 的 老婆 点 了 冷面 ， 第一次 吃 这样 的 ， 感觉 不错 啊 ！

Row 1:
Original: 昨天晚上想找个安静的环境和同事谈工作，去了绿茵阁。我是第一次到该店，一进去感觉很温馨，环境优雅。刚坐下，服务员倒了两杯水，但是与我们交流太少，默默倒玩完水就走了，可以给我们介绍一下“这是咱们餐前为您准备的柠檬水”啥的，很想跟服务员聊上两句的。需要改进的是，从服务员的角度来看，热情度不够，微笑比较少，问好声太少，服务行业就是要多与我们消费者寒暄与互动，昨晚变天了有点冷，服务员把我们安排在后面坐，这点比较好。从口味来看，我个人感觉比豪客来好吃，尤其是招牌牛肉，可惜的是服务员没有征询我们的意见，做成几分熟，我的同事觉得她的那份熟过头了，这点儿可以改进的。最近还出来了火锅，有时间再去尝尝火锅如何。最后，服务员让我们写个评价，写口味服务之类的，我同事简单写了一句，服务员说可以写详细点，写个服务好之类的……这很危

/home/vscode/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vscode/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vscode/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vscode/.local/li